In [1]:
import pandas as pd
import numpy as np
import nltk

In [96]:
from IPython.display import display_html
from nltk.corpus import stopwords

### Diccionario de stopWords para limpieza de texto

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/nico/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# La creacion de la lista de stopsWords customizada se debe a que en cada topico existe un grupo de palabras que no 
# aportan informacion relevante para el analisis, en este caso, por ejemplo, la palabra wine, no tiene relevancia
# ya es un pablara que hace referencia al objeto de analisis direcamente sin destacar niguna propiedad del el
customStopWords = ['wine', 'flavors', 'flavor']

In [5]:
#Stop Words que incluyen por defecto la libreria NLTK
stopWords = stopwords.words('english') + customStopWords

### Set de Datos

Los datos fueron tomados de sitio https://www.kaggle.com
Publicados en https://www.kaggle.com/zynicide/wine-reviews 


In [6]:
# df = pd.read_csv('../input/winemag-data-130k-v2.csv')
df = pd.read_csv('../data/winemag-data-130k-v2.csv', index_col = None)
df.drop(columns=['Unnamed: 0'], inplace = True)

Before the exploratory analis I will make a simply data transformation. All descriptions to lower case.

In [77]:
df.description = df.description.apply(lambda x : x.lower())
df.variety = df.variety.apply(lambda x : x.lower())

## Exploratory analysis

In [151]:
# This is a function that I found in stackOverFlow and it alow me to visualize various dataFrame's 
# at once

# def mydisplay(dfs, names=[]):
#     html_str = ''
#     if names:
#         html_str += ('<tr>' + 
#                      ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
#                      '</tr>')
#     html_str += ('<tr>' + 
#                  ''.join(f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>' 
#                          for df in dfs) + 
#                  '</tr>')
#     html_str = f'<table>{html_str}</table>'
#     html_str = html_str.replace('table','table style="display:inline"')
#     display_html(html_str, raw=True)

def mydisplay(dfs, names=[]):

    count = 0
    maxTables = 6
    
    if not names:
        names = [x for x in range(len(dfs))]

    html_str = ''
    html_th = ''
    html_td = ''
    
    for df, name in zip(dfs, names):
        if count <= (maxTables):
            html_th += (''.join(f'<th style="text-align:center">{name}</th>'))
            html_td += (''.join(f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>'))
            count += 1
        else:
            html_str += f'<tr>{html_th}</tr><tr>{html_td}</tr>'
            html_th = f'<th style="text-align:center">{name}</th>'
            html_td = f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>'
            count = 0
    
    
    if count != 0:
        html_str += f'<tr>{html_th}</tr><tr>{html_td}</tr>'
        
    
    html_str += f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

Let´s see what we have.

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22387 entries, 4 to 129950
Data columns (total 13 columns):
country                  22387 non-null object
description              22387 non-null object
designation              22387 non-null object
points                   22387 non-null int64
price                    22387 non-null float64
province                 22387 non-null object
region_1                 22387 non-null object
region_2                 22387 non-null object
taster_name              22387 non-null object
taster_twitter_handle    22387 non-null object
title                    22387 non-null object
variety                  22387 non-null object
winery                   22387 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 2.4+ MB


In [79]:
#Sample data of all columns
df.sample(1)

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
122240,US,"vanilla, cherry and a crunch of barbecue chips...",Watson Vineyard Old Vine,88,44.0,California,Dry Creek Valley,Sonoma,Virginie Boone,@vboone,Cast 2013 Watson Vineyard Old Vine Zinfandel (...,zinfandel,Cast


In [80]:
#Amount of uniques varieties
print('Se encuentrar ',df.variety.unique().size, ' variedades diferentes')
print('Hay vinos de ',df.country.unique().size, ' paises', end=' ')
print('y',df.winery.unique().size, ' bodegas diferentes')

Se encuentrar  183  variedades diferentes
Hay vinos de  1  paises y 2791  bodegas diferentes


Top 20 variatales mas revisados

In [81]:
pd.DataFrame(df.variety.value_counts()[:20]).reset_index(drop=False).rename(columns={'index':'varietal','variety':'recuento'})

,varietal,recuento
0,pinot noir,4788
1,chardonnay,2407
2,cabernet sauvignon,2372
3,red blend,1803
4,syrah,1678
5,zinfandel,1114
6,bordeaux-style red blend,1112
7,merlot,597
8,sauvignon blanc,574
9,rhône-style red blend,516


### Revision de los nombres de las variedades de los vinos

In [82]:
cleanNan = df.copy()
cleanNan.dropna(how='any', inplace=True)

In [106]:
#Palabras econtradas en los varietales
df1 = pd.DataFrame(pd.Series(' '.join([w for w in cleanNan.variety]).split()).value_counts()[:20]).reset_index(drop=False).rename(columns={'index':'words',0:'recuento'})

In [135]:
df1

,words,recuento
0,pinot,5173
1,noir,4801
2,blend,4274
3,red,3431
4,sauvignon,3006
5,cabernet,2749
6,chardonnay,2407
7,syrah,1678
8,bordeaux-style,1138
9,zinfandel,1114


In [134]:
#Varietales que contienen las palabras mas usadas
dfs = list()
for w in df1.words:
    word = w
    data = pd.DataFrame(pd.Series([x for x in cleanNan.variety if w in x]).value_counts()).reset_index(drop=False).rename(columns={'index':'word {}'.format(w),0:'recuento'})
    
    dfs.append((data,word))

****************************************************

In [152]:
#Varietals that contains the most common words
# mydisplay([d[0] for d in dfs], [w[1] for w in dfs])
mydisplay([d[0] for d in dfs])

0 1 2 3 4 5 6 
 
 
 word pinot 
 recuento 
 
 
 
 
 pinot noir 
 4788 
 
 
 pinot gris 
 269 
 
 
 pinot blanc 
 57 
 
 
 pinot grigio 
 52 
 
 
 pinot-chardonnay 
 5 
 
 
 pinot meunier 
 2 
 
 
 pinotage 
 2 
 
 
 pinot noir-syrah 
 2 
 
 
 pinot auxerrois 
 1 
 
 
 pinot gris-gewürztraminer 
 1 
 
 
 chardonnay-pinot blanc 
 1 
 
 
 pinot blanc-pinot noir 
 1 
 
 
 
 
 
 word noir 
 recuento 
 
 
 
 
 pinot noir 
 4788 
 
 
 gamay noir 
 9 
 
 
 grenache noir 
 2 
 
 
 pinot noir-syrah 
 2 
 
 
 baco noir 
 1 
 
 
 pinot blanc-pinot noir 
 1 
 
 
 
 
 
 word blend 
 recuento 
 
 
 
 
 red blend 
 1803 
 
 
 bordeaux-style red blend 
 1112 
 
 
 rhône-style red blend 
 516 
 
 
 white blend 
 334 
 
 
 sparkling blend 
 299 
 
 
 rhône-style white blend 
 154 
 
 
 bordeaux-style white blend 
 26 
 
 
 cabernet blend 
 11 
 
 
 champagne blend 
 10 
 
 
 tempranillo blend 
 5 
 
 
 grenache blend 
 3 
 
 
 touriga nacional blend 
 1 
 
 
 
 
 
 word red 
 recuento 
 
 
 
 
 red blend 
 1803 
 
 
 bordeaux-style red blend 
 1112 
 
 
 rhône-style red blend 
 516 
 
 
 
 
 
 word sauvignon 
 recuento 
 
 
 
 
 cabernet sauvignon 
 2372 
 
 
 sauvignon blanc 
 574 
 
 
 cabernet sauvignon-syrah 
 38 
 
 
 sauvignon blanc-semillon 
 24 
 
 
 syrah-cabernet sauvignon 
 21 
 
 
 cabernet sauvignon-merlot 
 18 
 
 
 semillon-sauvignon blanc 
 16 
 
 
 merlot-cabernet sauvignon 
 9 
 
 
 cabernet sauvignon-cabernet franc 
 5 
 
 
 cabernet sauvignon-sangiovese 
 4 
 
 
 sangiovese-cabernet sauvignon 
 3 
 
 
 tempranillo-cabernet sauvignon 
 2 
 
 
 cabernet sauvignon-malbec 
 1 
 
 
 malbec-cabernet sauvignon 
 1 
 
 
 
 
 
 word cabernet 
 recuento 
 
 
 
 
 cabernet sauvignon 
 2372 
 
 
 cabernet franc 
 284 
 
 
 cabernet sauvignon-syrah 
 38 
 
 
 syrah-cabernet sauvignon 
 21 
 
 
 cabernet sauvignon-merlot 
 18 
 
 
 cabernet blend 
 11 
 
 
 merlot-cabernet sauvignon 
 9 
 
 
 cabernet franc-merlot 
 9 
 
 
 merlot-cabernet 
 5 
 
 
 cabernet sauvignon-cabernet franc 
 5 
 
 
 merlot-cabernet franc 
 4 
 
 
 cabernet sauvignon-sangiovese 
 4 
 
 
 sangiovese cabernet 
 4 
 
 
 syrah-cabernet 
 3 
 
 
 cabernet-syrah 
 3 
 
 
 cabernet merlot 
 3 
 
 
 sangiovese-cabernet sauvignon 
 3 
 
 
 tempranillo-cabernet sauvignon 
 2 
 
 
 cabernet sauvignon-malbec 
 1 
 
 
 malbec-cabernet sauvignon 
 1 
 
 
 
 
 
 word chardonnay 
 recuento 
 
 
 
 
 chardonnay 
 2407 
 
 
 pinot-chardonnay 
 5 
 
 
 chardonnay-semillon 
 3 
 
 
 chardonnay-viognier 
 3 
 
 
 viognier-chardonnay 
 2 
 
 
 chardonnay-pinot blanc 
 1 
 
 
 semillon-chardonnay 
 1 
 
 
 7 8 9 10 11 12 13 14 
 
 
 word syrah 
 recuento 
 
 
 
 
 syrah 
 1678 
 
 
 cabernet sauvignon-syrah 
 38 
 
 
 syrah-grenache 
 22 
 
 
 syrah-cabernet sauvignon 
 21 
 
 
 grenache-syrah 
 17 
 
 
 syrah-petite sirah 
 7 
 
 
 syrah-mourvèdre 
 7 
 
 
 syrah-cabernet 
 3 
 
 
 syrah-viognier 
 3 
 
 
 cabernet-syrah 
 3 
 
 
 sangiovese-syrah 
 3 
 
 
 syrah-tempranillo 
 2 
 
 
 pinot noir-syrah 
 2 
 
 
 mourvèdre-syrah 
 1 
 
 
 tempranillo-syrah 
 1 
 
 
 syrah-grenache-viognier 
 1 
 
 
 tannat-syrah 
 1 
 
 
 syrah-merlot 
 1 
 
 
 syrah-petit verdot 
 1 
 
 
 
 
 
 word bordeaux-style 
 recuento 
 
 
 
 
 bordeaux-style red blend 
 1112 
 
 
 bordeaux-style white blend 
 26 
 
 
 
 
 
 word zinfandel 
 recuento 
 
 
 
 
 zinfandel 
 1114 
 
 
 
 
 
 word blanc 
 recuento 
 
 
 
 
 sauvignon blanc 
 574 
 
 
 pinot blanc 
 57 
 
 
 chenin blanc 
 57 
 
 
 grenache blanc 
 51 
 
 
 sauvignon blanc-semillon 
 24 
 
 
 semillon-sauvignon blanc 
 16 
 
 
 fumé blanc 
 14 
 
 
 vidal blanc 
 5 
 
 
 ugni blanc 
 3 
 
 
 muscat blanc 
 2 
 
 
 pinot blanc-pinot noir 
 1 
 
 
 chardonnay-pinot blanc 
 1 
 
 
 garnacha blanca 
 1 
 
 
 muscat blanc à petits grains 
 1 
 
 
 
 
 
 word rhône-style 
 recuento 
 
 
 
 
 rhône-style red blend 
 516 
 
 
 rhône-style white blend 
 154 
 
 
 
 
 
 word merlot 
 recuento 
 
 
 
 
 merlot 
 597 
 
 
 cabernet sauvignon-merlot 
 18 
 
 
 mer

************************************************

In [ ]:
df.variety.split()

Seria interesante ver que palabras en comun tienen las reviciones de distintos vinos. Eso nos daria una pauta de la similaridad de dichos varietales.

Hacer una lista de todas las palabras que aparecen y contarlas.
Tomar cada palabra de la lista y ver en cuantas variedades aparece
TOmar solo las palabras que se repiten en variedades diferentes
ver cuales son esas palabras
repeticiones

In [11]:
desc = df[['description', 'variety']].copy()
desc.description =  desc.description.apply(lambda x : ' '.join([w for w in x.split() if w.lower() not in stopWords]))

In [12]:
print('Numero medio de palabras con las que se describen los varietales')
round(desc.description.apply(lambda x : len(x.split())).mean(), 2)

Numero medio de palabras con las que se describen los varietales


24.43

Vamos a evaluar un varietal, Pinot Noir. Vamos a tomar todas las descripciones y hacer un lista de las palabras que se usaron

In [47]:
pinotNoir = pd.Series((' '.join([x for x in desc.groupby('variety').get_group('Pinot Noir').description]).split()))

In [48]:
print('Evaluacion de las descripciones del varital Pinot Noir')
print('Cantidad de palabras distintas usadas :', pinotNoir.unique().size)
print('Aparicion promedio de palabras : ', round(pinotNoir.value_counts().mean(), 2))
print('Lista ordenada de las palabras usadas :')
pinotNoir.value_counts()[:5]

Evaluacion de las descripciones del varital Pinot Noir
Cantidad de palabras distintas usadas : 23603
Aparicion promedio de palabras :  14.26
Lista ordenada de las palabras usadas :


cherry    3952
Pinot     3434
fruit     3415
red       2765
black     2459
dtype: int64

El proximo paso deberia ser empezar a ver como se repite el uso de las palabras cuando se describen diferentes vinos.
Una palabra que se usan en la descripcion de varietales diferentes de manera repetida es una palabra que tendria bajo 
peso descriptivo, podriamos decir que esa palabra no describe muy bien a la clase de vino.
En el lado opuesto encontrariamos a las palabras que, son adejetivos, que aparecen varias veces en las descripciones
pero lo hacen solo para una o dos variedades especificas que al mismo tiempo tendrian algo en comun,
podriamos decir que esas palabras describen muy bien un varietal o una propiedad bastante particular del varietal

***************************************

***************************************

In [ ]:
#What varieties are the most described or tested by the wine taster
count = 0
for l in df.variety.value_counts()>1000:
    if l:
        count += 1
print (count , '  Are de varieties with more than 1000 descriptions')

Ok, I can´t work with more than 100 topics. So I will work with the most relevant varieties making a set of 28 topics to analyse.

**DATA - BAG OF WORDS**

In [ ]:
data = list(df.description.values)

**Term frequency – Inverse document frequency**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer

In [ ]:
no_features = 1000

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df=2, max_df=0.95, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

Parameters **min_df** and **max_df** are the minimum and maximum number of apparition of one particular word to be counted in the algorithm also I am using StopWords to reduce the noise in the output.

** Float numbers are in percent and integer are absolute apparition.*

**CountVectorizer**

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(data)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
from sklearn.decomposition import NMF , LatentDirichletAllocation

In [ ]:
no_topic = 28

In [ ]:
#for NMF

nmf = NMF(n_components = no_topic , random_state=1 , alpha=.1 , l1_ratio=.5 , init='nndsvd').fit(tfidf)


#for LDA

lda = LatentDirichletAllocation(n_components=no_topic , max_iter=10 , learning_method='online' , learning_offset=50. , random_state=0)

lda_v = lda.fit(tf)

**First topics**

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda_v, tf_feature_names, no_top_words)

**pyLDAvis for visualization**

In [ ]:
import pyLDAvis.sklearn

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='tsne')

In [ ]:
panel

Using this tool we can vary the lambda parameter to observe how the terms are grouped and their frequency of appearancem

**So, the next step is to interpret the topics to reduce the dimensionality of the dataset and group the most similar varieties.**

